<a href="https://colab.research.google.com/github/sergmiller/vk-cup-final-2021/blob/main/sergmiller_RuGPT_bundle_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers sentencepiece
!wget https://storage.yandexcloud.net/vk-cup-data/test_pipeline_questions.csv 

     |████████████████████████████████| 2.6 MB 14.2 MB/s 
     |████████████████████████████████| 1.2 MB 55.5 MB/s 
     |████████████████████████████████| 636 kB 57.9 MB/s 
     |████████████████████████████████| 3.3 MB 68.8 MB/s 
     |████████████████████████████████| 895 kB 88.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
--2021-08-21 20:24:31--  https://storage.yandexcloud.net/vk-cup-data/test_pipeline_questions.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250 [text/csv]
Saving to: ‘test_pipeline_questions.csv’

test_pipeline_quest 100%[===================>]     250  --.-KB/s    in 0s      

2021-08-21 20:24:31 (62.7 MB/s) - ‘test_pipeline_questions.csv’ saved [250/250

In [3]:
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
from google.colab import drive
drive.mount('/gdrive')

MOUNT_DIR = "/gdrive/My Drive/Colab Notebooks/ColabMountDir/"

Mounted at /gdrive


In [6]:
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)#.cuda()

In [10]:
def calc_likelihood(text):
    tokens_ids = tokenizer.encode(text, return_tensors="pt")#.cuda()
    prob = model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood

In [8]:
tokens_ids = tokenizer.encode("1 2 3", return_tensors="pt")#.cuda()
prob = model.forward(tokens_ids, output_hidden_states=True)

In [9]:
logits = prob.logits.cpu().detach().numpy()
hidden = prob.hidden_states
logits.shape, len(hidden), hidden[0].shape

((1, 3, 50257), 25, torch.Size([1, 3, 1536]))

In [37]:
def make_bundle(question: str, answers: list) -> dict:
  torch2np = lambda x: x.cpu().detach().numpy()
  def single_sent_info(sent: str) -> dict:
    tokens_ids = tokenizer.encode(sent, return_tensors="pt").cuda()
    prob = model.forward(tokens_ids,output_hidden_states=True)
    logits = prob.logits[0]
    hidden = prob.hidden_states
    T = len(tokens_ids[0])
    assert list(logits.shape) == [T, 50257], logits.shape
    assert len(hidden) == 25 and list(hidden[-1].shape) == [1, T, 1536], hidden[0].shape
    state = dict()
    state["tokens"] = torch2np(tokens_ids)
    tokens = tokens_ids[0].cpu().tolist()
    logits = torch2np(logits)
    state["likelihood"] = np.sum([logits[i, j] for i, j in enumerate(tokens)])
    state["embed"] = torch2np(hidden[-1])
    return state
  bundle = dict()
  bundle["q"] = single_sent_info(question)
  bundle["a1"] = single_sent_info(question + " " + answers[0])
  bundle["a2"] = single_sent_info(question + " " + answers[1])
  bundle["a3"] = single_sent_info(question + " " + answers[2])
  return bundle

In [8]:
question = "Кто доказал теорию эволюции?"
answers = ["Даня Милохин", "Чарльз Дарвин", "Ленардо Да Винчи"]
def predict(question, answers):
  scores = [calc_likelihood(question + " " +  answer) for answer in answers]
  return str(np.argmax(scores))

# predict(question, answers)
b = make_bundle(question, answers)

In [9]:
b["a1"]["likelihood"]

40.67775

In [11]:
train_data = pd.read_csv(MOUNT_DIR + 'rus_train_dataset.csv', encoding='utf-8', sep='|')
en_ru_aux_data = pd.read_csv(MOUNT_DIR + 'en_to_rus_train_dataset.csv', encoding='utf-8', sep='|')

Mounted at /gdrive


In [13]:
train_data

,Unnamed: 0,question,ps_0,ps_1,ps_2,right_answer_id
0,0,Как называется половина основного времени матч...,Тайм,Период,Гейм,0
1,1,Какая из указанных команд чаще других становил...,Россия,Англия,Чехия,0
2,2,В каком году мужская сборная СССР по баскетбол...,1972,1980,1988,0
3,3,"Как называется технология, используемая для бе...",NFC,Android Pay,Apple Pay,0
4,4,Где в 2014 году проходил Чемпионат мира по фут...,Бразилия,Новая Зеландия,Германия,0
...,...,...,...,...,...,...
4056,4056,"Как называются рецепторы в организме человека,...",Хеморецепторы,Барорецепторы,Механорецепторы,1
4057,4057,Какая из перечисленных стран НЕ входила в сост...,Казахстан,Украина,Финляндия,2
4058,4058,В каком году была выпущена приставка «Playstat...,2000,2001,1999,0
4059,4059,Как называлась добыча мёда диких пчёл на Руси?,Бортничество,Капище,Полюдье,0


In [12]:
en_ru_aux_data

,Unnamed: 0,question,ps_0,ps_1,ps_2,right_answer_id
0,0,Одним из остатков этого государства прежний ст...,Оклахома,Флорида,Луизиана,2.0
1,1,Самое большое озеро в этом штате - озеро окехо...,Гавайи,Теннесси,Флорида,2.0
2,2,Какая государственная столица Вашингтона?,Олимпия,Сиэтл,Walla Walla,0.0
3,3,Какова государственная столица Калифорнии?,Сакраменто,Лос-Анджелес,Сан-Диего,0.0
4,4,Что такое государственная столица Айдахо?,Бойсе,Айдахо-Фолс,Покателло,0.0
...,...,...,...,...,...,...
1936,1948,Три собаки и два щенка весят 128 фунтов. Четыр...,32 фунта,16 фунтов,80 фунтов,2.0
1937,1949,"Изабелла взяла тест, который имел 20 вопросов....",10,5,3,1.0
1938,1950,"Какие две планеты солнечной системы, отличающи...",Меркурий и Марс,Меркурий и Венеры,Венера и Юпитер,1.0
1939,1951,"Эта луна, подшипщая название паромных мертвых ...",Charon,Nereid,Proteus,0.0


In [49]:
import tqdm
import json
import os

def prepare_dataset_bundle(df: pd.DataFrame)->list:
  # if not os.path.exists(dirpath):
  #   os.mkdir(dirpath)
  res = []
  for line in tqdm.tqdm(df.values, position=0):
    idx, q, a1, a2, a3, _ = line
    bundle = make_bundle(q, [str(a1), str(a2), str(a3)])
    # np.save(dirpath + "/" + str(idx), bundle)
    res.append(bundle)
  return res

In [77]:
train_bundle = prepare_dataset_bundle(train_data)

100%|██████████| 4061/4061 [16:39<00:00,  4.06it/s]


In [78]:
np.save(MOUNT_DIR + "train_dataset_bundle", train_bundle)

In [14]:
aux_dataset_bundle = prepare_dataset_bundle(en_ru_aux_data)

100%|██████████| 1941/1941 [09:19<00:00,  3.47it/s]


In [17]:
np.save(MOUNT_DIR + "en_ru_aux_dataset_bundle", aux_dataset_bundle)

In [ ]:
def create_submission(questions_csv_path):
  data = pd.read_csv('test_pipeline_questions.csv', encoding='utf-8', sep='|')
  answers = [predict(data['question'][i],
                     [data['ps_0'][i], data['ps_1'][i], data['ps_2'][i]]) for i in range(len(data))]
  
  with open('answer.txt', 'w') as f:
    f.write('\n'.join(answers))
  return answers

In [49]:
data = pd.read_csv('test_pipeline_questions.csv', encoding='utf-8', sep='|')

In [50]:
data.head()

,question,ps_0,ps_1,ps_2
0,В каком году был создан ВКонтакте?,1929,2021,2006
1,Кто придумал теорию относительности?,Альберт Энштейн,Исаак Ньютон,Мирон Федоров


Перевод обработанного датасета http://nlp.cs.washington.edu/triviaqa/ (предобработку см в triviaQA_large_preparation.ipynb): здесь будет перевод с сэмплированием негативных ответов(т к в оригинальном датасете их нет)

In [39]:
import sentencepiece

In [5]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
import torch
mname = "facebook/wmt19-en-ru"
tokenizer_tr = FSMTTokenizer.from_pretrained(mname)
model_tr = FSMTForConditionalGeneration.from_pretrained(mname)

# from transformers import AutoTokenizer, AutoModelForPreTraining
  
# tokenizer_tr = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")

# model_tr = AutoModelForPreTraining.from_pretrained("cointegrated/LaBSE-en-ru")

def pad_sequences(batch, pad_value=2):
    n = max(len(v) for v in batch)
    return torch.tensor([v + [pad_value] * (n - len(v)) for v in batch])

input = "Machine learning is great, isn't it?"
input_ids = tokenizer_tr.encode(input)
input2 = "Machine learning is great!"
input_ids2 = tokenizer_tr.encode(input2)
outputs = model_tr.generate(pad_sequences([input_ids, input_ids2]), num_beams=1)
print(outputs.shape)
decoded = tokenizer_tr.decode(outputs[0], skip_special_tokens=True)
print(decoded) # Машинное обучение - это здорово, не так ли?
decoded = tokenizer_tr.decode(outputs[1], skip_special_tokens=True)
print(decoded) # Машинное обучение - это здорово, не так ли?

torch.Size([2, 14])
Машинное обучение - это здорово, не так ли?
Машинное обучение - это здорово!


In [24]:
%%time
outputs = model_tr.generate(pad_sequences([input_ids, input_ids2]))

CPU times: user 1.92 s, sys: 14.9 ms, total: 1.93 s
Wall time: 1.92 s


In [28]:
%%time
outputs = model_tr.generate(pad_sequences([input_ids, input_ids2]),  num_beams=1)

CPU times: user 1.3 s, sys: 3.77 ms, total: 1.3 s
Wall time: 1.3 s


In [6]:
tr_cache = dict()

def translate(text: str) -> str:
  if text in tr_cache:
    return tr_cache[text]
  input_ids = tokenizer_tr.encode(text, return_tensors="pt")
  outputs = model_tr.generate(input_ids)
  decoded = tokenizer_tr.decode(outputs[0], skip_special_tokens=True)
  tr_cache[text] = decoded
  return decoded

def translate_batch(texts: list) -> str:
  res = [None for _ in range(len(texts))]
  others = []
  tokenized = []
  for i, t in enumerate(texts):
    if t in tr_cache:
      res[i] = tr_cache[t]
    else:
      others.append(i)
      tokenized.append(tokenizer_tr.encode(t))
  if len(others) == 0:
    return res
  outputs = model_tr.generate(pad_sequences(tokenized))
  for idx, out in zip(others, outputs):
    decoded = tokenizer_tr.decode(out, skip_special_tokens=True)
    tr_cache[texts[idx]] = decoded
    res[idx] = decoded
  return res

In [10]:
en_trivia_qa = pd.read_csv(MOUNT_DIR + 'en_questions_bundle.csv')

In [11]:
en_trivia_qa.head()

,Unnamed: 0,0,1
0,0,Where in England was Dame Judi Dench born?,York
1,1,From which country did Angola achieve independ...,Portugal
2,2,Which city does David Soul come from?,Chicago
3,3,Who won Super Bowl XX?,Chicago Bears
4,4,Which was the first European country to abolis...,Norway


In [12]:
en_trivia_qa.shape

(58018, 3)

In [13]:
qids = np.arange(len(en_trivia_qa))
np.random.seed(0)
np.random.shuffle(qids)
print(qids[:10])

[17114 35183 46480 12596 55446 18361 56939 15779  3745 44208]


In [14]:
SAMPLE_SIZE = 2048 * 2

In [46]:
import tqdm

batch_size = 16
qs_translated = []
for batch in tqdm.tqdm(np.arange(0, SAMPLE_SIZE, batch_size), position=0):
  qbatch = qids[batch:batch + batch_size]
  qs = en_trivia_qa.values[qbatch, 1]
  qs_tr = translate_batch(qs)
  qs_translated.extend(qs_tr)

In [16]:
# np.save(MOUNT_DIR + "qs_translated_4K", qs_translated)
qs_translated = np.load(MOUNT_DIR + "qs_translated_4K.npy", allow_pickle=True)

In [17]:
qs_translated[:5]

array(['Компания Loake Brothers, основанная в 1880 году, известна своим высоким качеством.',
       '"Кто играет главную роль в фильме" Бэтмен "?',
       'Под каким названием проходят съемки фильма "Лед и пламя"?',
       'По словам его создателя, какой известный вымышленный персонаж, в свою очередь, лгал о своем возрасте, чтобы попасть в резерв Королевского военно-морского флота во время Второй мировой войны, из которого он вышел в качестве командира?',
       'Кто такой "Розовая пантера"?'], dtype='<U1378')

In [ ]:
import tqdm

NEGATIVES = 10

qat_candidates = []

np.random.seed(0)

for i in tqdm.tqdm(np.arange(2 * SAMPLE_SIZE), position=0):
    idx = qids[i]
    negids = np.random.choice(qids, NEGATIVES)
    lls = []
    q_tr = qs_translated_all[i]
    ra = en_trivia_qa.values[idx][2]
    answers = [ra,]
    for neg in negids:
        if neg == idx:
            continue
        a = en_trivia_qa.values[neg][2]
        answers.append(a)
    answers_tr = translate_batch(answers)
    qat_candidates.append((q_tr, answers_tr))

In [18]:
qat_candidates = np.load(MOUNT_DIR + "en_triviaQA_translated_qat_candidates_4K.npy", allow_pickle=True)

In [29]:
qat_candidates[0]

array(['Компания Loake Brothers, основанная в 1880 году, известна своим высоким качеством.',
       list(['Обувь', 'Виктория Азаренка', 'Браун', 'Баптистская церковь Уэстборо', 'The Lion King', 'Ремесло', 'Дания', 'Эритреа', 'Пролив Кука', 'Yango', 'Бостон'])],
      dtype=object)

In [34]:
import tqdm

NEGATIVES = 10

qat_pairs = []

np.random.seed(0)

for i in tqdm.tqdm(np.arange(SAMPLE_SIZE), position=0):
  idx = qids[i]
  lls = []
  q_tr = qs_translated[i]
  ra_tr = qat_candidates[i][1][0]
  for a_neg_tr in qat_candidates[i][1][1:]:
    _ll = calc_likelihood(q_tr + " " +  a_neg_tr)
    lls.append((_ll, a_neg_tr))
  lls = sorted(lls, key=lambda x:x[0])
  hard_neg_1_a_tr = lls[-1][1]
  hard_neg_2_a_tr = lls[-2][1]
  order = np.random.permutation(3)
  sample = [
            (q_tr, ra_tr, 1),
            (q_tr, hard_neg_1_a_tr, 0),
            (q_tr, hard_neg_2_a_tr, 0),
  ]
  qat_pairs.append(sample[order[0]])
  qat_pairs.append(sample[order[1]])
  qat_pairs.append(sample[order[2]])

100%|██████████| 4096/4096 [23:42<00:00,  2.88it/s]


In [35]:
# np.save(MOUNT_DIR + "en_triviaQA_translated_with_negatives_qat_pairs_4K.npy", qat_pairs)

In [36]:
qat_pairs[:6]

[('Компания Loake Brothers, основанная в 1880 году, известна своим высоким качеством.',
  'The Lion King',
  0),
 ('Компания Loake Brothers, основанная в 1880 году, известна своим высоким качеством.',
  'Баптистская церковь Уэстборо',
  0),
 ('Компания Loake Brothers, основанная в 1880 году, известна своим высоким качеством.',
  'Обувь',
  1),
 ('"Кто играет главную роль в фильме" Бэтмен "?', 'Уиттингтон, Ричард', 0),
 ('"Кто играет главную роль в фильме" Бэтмен "?', 'Мишель Баффер', 1),
 ('"Кто играет главную роль в фильме" Бэтмен "?',
  'Ошибки, допущенные в ходе выборов, могут привести к тому, что многие избиратели не смогут проголосовать за кандидата в президенты США от Республиканской партии Дональда Трампа, который, по их мнению, является наиболее вероятным кандидатом от Республиканской партии на предстоящих президентских выборах в 2016 году.',
  0)]

In [46]:
def pairs2dataset(arr: np.array) -> pd.DataFrame:
  res = []
  # idx, q, a1, a2, a3, _ = line
  idx = 0
  for i in np.arange(0, len(arr), 3):
    q = arr[i][0]
    a1 = arr[i][1]
    a2 = arr[i+1][1]
    a3 = arr[i+2][1]
    t = np.argmax([arr[i][2], arr[i+1][2], arr[i+2][2]])
    res.append([idx, q, a1, a2, a3, t])
    idx += 1
  return pd.DataFrame(res)

In [47]:
en_ru_4k_dataset = pairs2dataset(qat_pairs)

In [48]:
en_ru_4k_dataset.head()

,0,1,2,3,4,5
0,0,"Компания Loake Brothers, основанная в 1880 год...",The Lion King,Баптистская церковь Уэстборо,Обувь,2
1,1,"""Кто играет главную роль в фильме"" Бэтмен ""?","Уиттингтон, Ричард",Мишель Баффер,"Ошибки, допущенные в ходе выборов, могут приве...",1
2,2,"Под каким названием проходят съемки фильма ""Ле...",Игра престолов,Жан Симмонс,Мэри Квант,0
3,3,"По словам его создателя, какой известный вымыш...","Троллоп, Энтони",Джеймс Бонд,Мария (мать Иисуса),1
4,4,"Кто такой ""Розовая пантера""?",Улус-часы,"В то же время, несмотря на то, что в настоящее...","Манчини, Генри",2


In [50]:
en_ru_4k_bundle = prepare_dataset_bundle(en_ru_4k_dataset)

100%|██████████| 4096/4096 [10:15<00:00,  6.66it/s]


In [53]:
np.save(MOUNT_DIR + "en_ru_4k_dataset", en_ru_4k_dataset)

In [52]:
np.save(MOUNT_DIR + "en_ru_4k_bundle", en_ru_4k_bundle)

In [17]:
def make_pairs_with_negatives(qa_bundle: np.array, with_bundle:bool=False) -> np.array:
    import tqdm

    NEGATIVES = 10

    qat_pairs = []

    np.random.seed(0)

    for i in tqdm.tqdm(np.arange(qa_bundle.shape[0]), position=0):
        lls = []
        q = qa_bundle[i][0]
        ra = qa_bundle[i][1]
        negids = np.random.choice(np.arange(qa_bundle.shape[0]), NEGATIVES)
        lls = []
        answers = []
        for neg in negids:
            if neg == i:
                continue
            a_neg = qa_bundle[neg][1]
            _ll = calc_likelihood(q + " " +  a_neg)
            lls.append((_ll, a_neg))
        lls = sorted(lls, key=lambda x:x[0])
        hard_neg_1_a = lls[-1][1]
        hard_neg_2_a = lls[-2][1]
        
        order = np.random.permutation(3)
        sents = np.array([ra, hard_neg_1_a, hard_neg_2_a])[order]
        b = None
        if with_bundle:
            b = make_bundle(q, sents)
        qat_pairs.append([q,] + list(sents) + [np.argmin(order), b])
    return qat_pairs

In [11]:
rubq_qa = pd.read_csv(MOUNT_DIR + "rubq_questions_bundle.csv")

In [13]:
rubq_qa.values

array([[0, 'Что может вызвать цунами?', 'Землетрясение'],
       [1, 'Кто написал роман «Хижина дяди Тома»?', 'Г. Бичер-Стоу'],
       [2, 'Кто автор пьесы «Ромео и Джульетта»?', 'Шекспир'],
       ...,
       [2327, 'В каком году сняли с производства Jaguar E-type?', '1974'],
       [2328,
        'С какого года закончили выпускать Rolls-Royce Silver Ghost?',
        '1926'],
       [2329, 'В каком году сняли с производства Porsche 550?', '1956']],
      dtype=object)

In [ ]:
rubq_qat_pairs = make_pairs_with_negatives(rubq_qa.values[:, 1:], True)

In [ ]:
np.save(MOUNT_DIR + "rubq_qat_pairs", rubq_qat_pairs)